In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv(r'2019-Oct.csv')

In [4]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
data.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Define the number of rows you want in the test data
num_rows = 5

# Generate test data
testdata = {
    'event_time': [datetime(2019, 10, 1) + timedelta(seconds=i) for i in range(num_rows)],
    'event_type': ['view', 'click', 'purchase', 'view', 'click'],
    'product_id': [random.randint(1000000, 9999999) for _ in range(num_rows)],
    'category_id': [random.randint(2000000000000000000, 2999999999999999999) for _ in range(num_rows)],
    'category_code': ['furniture.living_room.sofa', 'appliances.kitchen.fridge', 'electronics.tv', 'clothing.men.shirt', 'furniture.bedroom.bed'],
    'brand': ['IKEA', 'Samsung', 'Sony', 'Nike', 'Serta'],
    'price': [round(random.uniform(20.00, 1000.00), 2) for _ in range(num_rows)],
    'user_id': [random.randint(100000000, 999999999) for _ in range(num_rows)],
    'user_session': [str(uuid.uuid4()) for _ in range(num_rows)]
}

# Create DataFrame
df = pd.DataFrame(testdata)

# Save DataFrame to CSV file
df.to_csv("test_data.csv", index=False)

# Optionally, print the DataFrame to confirm
print(df)


NameError: name 'uuid' is not defined

In [6]:
%%writefile testutility.py
import logging
import pandas as pd
import yaml
import re

################
# File Reading #
################

def read_config_file(filepath):
    """
    Reads a YAML configuration file and returns the data.
    """
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)
            return None

def replacer(string, char):
    """
    Replace multiple occurrences of a character in a string with a single occurrence.
    """
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df, table_config):
    """
    Standardizes column names by:
    - Lowercasing all column names
    - Replacing non-word characters with underscores
    - Removing extra underscores
    - Checking column name and length validation against the YAML config
    """
    df.columns = df.columns.str.lower()  # Lowercase all column names
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)  # Replace non-word chars with '_'
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))  # Strip underscores at both ends
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))  # Remove extra underscores

    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()

    # Sort columns of the dataframe
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)

    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("Column name and column length validation passed.")
        return 1
    else:
        print("Column name and column length validation failed.")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Columns in file but not in YAML:", mismatched_columns_file)
        missing_yaml_file = list(set(expected_col).difference(df.columns))
        print("Columns in YAML but not in file:", missing_yaml_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


def convert_to_datetime(df, column_name):
    """
    Convert a specified column in the DataFrame to datetime format.
    """
    try:
        df[column_name] = pd.to_datetime(df[column_name])
        print(f"Successfully converted {column_name} to datetime.")
    except Exception as e:
        print(f"Error converting {column_name} to datetime: {e}")
    return df


def check_missing_values(df):
    """
    Check for missing values in the DataFrame.
    """
    missing_values = df.isnull().sum()
    print(f"Missing values:\n{missing_values}")
    return missing_values


def summary_statistics(df):
    """
    Generate summary statistics for the DataFrame.
    """
    stats = df.describe()
    print(f"Summary statistics:\n{stats}")
    return stats


Writing testutility.py


In [7]:
%%writefile config.yaml
file_type: csv
dataset_name: ecommerce_data
file_name: 2019-Oct
table_name: ecommerce_transactions
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 0
columns:
    - event_time
    - event_type
    - product_id
    - category_id
    - category_code
    - brand
    - price
    - user_id
    - user_session


Writing config.yaml


In [8]:
# Import the utility file
import testutility as util

# Read the config file
config_data = util.read_config_file("config.yaml")

# Access specific config data
inbound_delimiter = config_data['inbound_delimiter']
print(f"Inbound delimiter: {inbound_delimiter}")

# You can also access other elements similarly
dataset_name = config_data['dataset_name']
file_name = config_data['file_name']

print(f"Dataset Name: {dataset_name}")
print(f"File Name: {file_name}")


c:\Users\steph\Downloads\Data Glacier\Week_6\archive (4)\testutility.py:38: SyntaxWarning: invalid escape sequence '\w'
  df.columns = df.columns.str.replace('[^\w]', '_', regex=True)  # Replace non-word chars with '_'


Inbound delimiter: ,
Dataset Name: ecommerce_data
File Name: 2019-Oct


In [9]:
# Import the utility file
import testutility as util

# Read the config file
config_data = util.read_config_file("config.yaml")

# Inspect the entire content of the config file
config_data


{'file_type': 'csv',
 'dataset_name': 'ecommerce_data',
 'file_name': '2019-Oct',
 'table_name': 'ecommerce_transactions',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 0,
 'columns': ['event_time',
  'event_type',
  'product_id',
  'category_id',
  'category_code',
  'brand',
  'price',
  'user_id',
  'user_session']}

In [12]:
import pandas as pd

# Normal reading process
df_sample = pd.read_csv("2019-Oct.csv", delimiter=',')
df_sample.head()


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [13]:
import pandas as pd
import testutility as util

# Read the config file
config_data = util.read_config_file("config.yaml")

# Extract file type and file name from config
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'

# Print the full path of the file being read (optional)
print("Source file:", source_file)

# Read the file using the delimiter from the config file
df = pd.read_csv(source_file, delimiter=config_data['inbound_delimiter'])

# Show the first few rows of the dataframe
df.head()


Source file: ./2019-Oct.csv


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [14]:
# Validate the header of the file
validation_result = util.col_header_val(df, config_data)

# Print the result of the validation
if validation_result == 1:
    print("Header validation passed!")
else:
    print("Header validation failed!")


Column name and column length validation passed.
Header validation passed!


In [15]:
# Print the columns of the DataFrame
print("Columns of the file are:", df.columns)

# Print the columns defined in the YAML config file
print("Columns of the YAML are:", config_data['columns'])


Columns of the file are: Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')
Columns of the YAML are: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']


In [17]:
import os

# Perform header validation
validation_result = util.col_header_val(df, config_data)

if validation_result == 0:
    # Validation failed
    print("Validation failed")
    # Code to handle the failure, e.g., reject the file or stop further processing
    # You can remove or move the file to an error directory, log the issue, or raise an exception.
    # Example of moving the file to an error directory
    import shutil
    import os

    # Define the error directory
    error_dir = './error_files'
    os.makedirs(error_dir, exist_ok=True)

    # Move the file to the error directory
    shutil.move(source_file, os.path.join(error_dir, os.path.basename(source_file)))
    print(f"File moved to error directory: {os.path.join(error_dir, os.path.basename(source_file))}")

    # Optionally, you might also log the error or raise an exception
    # logging.error("Column validation failed. File has been moved to error directory.")
else:
    # Validation passed
    print("Column validation passed")
    # Code to perform further actions in the pipeline
    # This could include data processing, saving the file, or any other steps you need to take
    # Example of saving the DataFrame to a new file or performing some analysis
    processed_file_path = './processed_files/' + os.path.basename(source_file)
    os.makedirs(os.path.dirname(processed_file_path), exist_ok=True)
    df.to_csv(processed_file_path, index=False)
    print(f"File processed and saved to: {processed_file_path}")

    # Further actions can be included here
    # For example: data analysis, transformations, or saving to a database


Column name and column length validation passed.
Column validation passed
File processed and saved to: ./processed_files/2019-Oct.csv


In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import uuid

# Define the number of rows you want in the test data
num_rows = 5

# Generate test data
testdata = {
    'event_time': [datetime(2019, 10, 1) + timedelta(seconds=i) for i in range(num_rows)],
    'event_type': ['view', 'click', 'purchase', 'view', 'click'],
    'product_id': [random.randint(1000000, 9999999) for _ in range(num_rows)],
    'category_id': [random.randint(2000000000000000000, 2999999999999999999) for _ in range(num_rows)],
    'category_code': ['furniture.living_room.sofa', 'appliances.kitchen.fridge', 'electronics.tv', 'clothing.men.shirt', 'furniture.bedroom.bed'],
    'brand': ['IKEA', 'Samsung', 'Sony', 'Nike', 'Serta'],
    'price': [round(random.uniform(20.00, 1000.00), 2) for _ in range(num_rows)],
    'user_id': [random.randint(100000000, 999999999) for _ in range(num_rows)],
    'user_session': [str(uuid.uuid4()) for _ in range(num_rows)]
}

# Create DataFrame
df = pd.DataFrame(testdata)

# Save DataFrame to CSV file
df.to_csv("test_data.csv", index=False)

# Optionally, print the DataFrame to confirm
print(df)


           event_time event_type  product_id          category_id  \
0 2019-10-01 00:00:00       view     4552274  2703130504621449332   
1 2019-10-01 00:00:01      click     3248561  2312669330836483273   
2 2019-10-01 00:00:02   purchase     6591887  2397098276626259275   
3 2019-10-01 00:00:03       view     1395672  2552915327074752310   
4 2019-10-01 00:00:04      click     9495509  2578711066147386388   

                category_code    brand   price    user_id  \
0  furniture.living_room.sofa     IKEA  283.48  342030786   
1   appliances.kitchen.fridge  Samsung  422.00  214987970   
2              electronics.tv     Sony  518.75  862815439   
3          clothing.men.shirt     Nike  859.21  655123399   
4       furniture.bedroom.bed    Serta  187.93  503632536   

                           user_session  
0  daa55e5f-5b62-44b3-8b3b-83fcf6480044  
1  fa7c7375-f0a6-4762-b8b8-acb281a54eb3  
2  ff09e577-5594-44e0-8a10-a549d4a54037  
3  60307586-9d8e-4a63-9dc4-f3e46b9a92b1  
4  eb651